# Predictive Regression

## Imports

In [1]:
# <include-predictive_regression/utils.py>

In [1]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio

from predictive_regression import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"
# pio.base_renderers.default = "vscode"

from patsy import dmatrices
import statsmodels.api as sm

## Summary

In [44]:
df_data.index.levels[1]

Index(['BA', 'C', 'DD', 'F', 'GE', 'JPM', 'LOW', 'LUV', 'MAR', 'T', 'WFC',
       'XOM'],
      dtype='object', name='ticker')

In this assignment we create a two-stage regression model to predict the future weekly "returns" of five-year credit default swaps for 12 large-capitalization, publicly traded companies. The first entails creating contemporaneous models of the CDS "return" and stock price return and then calculating residuals. In the second stage, we create a model that predicts the CDS "return" residual based on the prior day's equity return residual. Our period of analysis spans from 2018-01-03 to 2018-4-30.

#### Contemporaneous Models

We model contemporaneous CDS "returns" as a function of stock price return and "return" an index of other CDS "returns", defined as the arithmetic average the CDS's of the other 11 companies in our universe. For each of these models we perform ordinary least squares regression for each ticker, for each contiguous 16 week period throughout our entire period of analysis. 

$$
r_{E}^{CDS} \sim r_{E}^{Equity} + r_{Index}^{CDS} + \epsilon
$$

We model contemporaneous equity returns as a function of the return on the market, defined as the return on SPY.

$$
r_{E}^{Equity} \sim r_{SPY}^{Equity} + \epsilon
$$

To predict the contemporaneous CDS return we end up with
$$
f_{E,n} = \beta_{E,n}^{Intercept} + \beta_{E,n}^{Equity} \cdot r_{E,n}^{Equity} + \beta_{E,n}^{Index} \cdot r_{E,n}^{Index}
$$

$$
g_{E,n} = \gamma_{E,n}^{Intercept} + \gamma_{E,n}^{SPY} \cdot r_{SPY,n}^{Equity}
$$

Contemporanous residuals can the be defined as for CDS
$$
\rho_{E,n} = r_{E,n}^{CDS} - f_{E,n}
$$

and

$$
c_{E,n} = r_{E,n}^{Equity} - g_{E,n}
$$

#### Predictive Model

Our predictive model then becomes

$$
\rho_{E,n} = c_{E,n-1} + \epsilon
$$

where we are lagging the equity residual back one observation from the CDS residual and using various window methodologies and lengths to calculate the regression coefficients.

Requirements:

* box_car and ewm regressions of various window lengths for cds_resid_n ~ equity_resid_n-1

Interesting things to try:

* r-squared overall for equity, cds and cds_resd ~ eq_resid models
* r_squared by date and by ticker for each model
* robust regressions for each model type

* end_to_end model with cds_return_n ~ cds_n-1, equity_n-1, spy_n-1, index_n-1
* transformer model for end to end integrated model

* across all tickers together as oppossed to individually

## Contemporaneous CAPM Model

In [2]:
date_range = pd.date_range("2018-01-03", "2021-04-30", freq="7D")
df_data = utils.get_data(date_range)
df_data.tail()

series              adj_close  r_equity  r_spread  spread5y   r_index  \
date       ticker                                                       
2021-04-28 LUV      61.770000 -0.004362 -0.017298  0.008339 -0.010192   
           MAR     149.450000  0.031885 -0.008381  0.008758 -0.011003   
           T        30.960000  0.027839 -0.023042  0.006872 -0.009670   
           WFC      44.983531  0.041673 -0.031137  0.005364 -0.008934   
           XOM      58.110000  0.036986  0.009787  0.003850 -0.012655   

series                r_spy  
date       ticker            
2021-04-28 LUV     0.003191  
           MAR     0.003191  
           T       0.003191  
           WFC     0.003191  
           XOM     0.003191

In [29]:
df_eq_errors, eq_summary = utils.get_errors(df_data, model="RLM")

In [35]:
df_cds_errors, cds_summary = utils.get_errors(df_data, model="RLM", formula="r_spread ~ r_equity + r_index + 1")

In [36]:
df_resid = pd.concat([df_cds_errors.resid, df_eq_errors.resid.groupby("ticker").shift()], axis=1)
df_resid.columns = ["cds_resid", "eq_resid"]
df_resid = df_resid.unstack("ticker")
df_resid

cds_resid                                                    \
ticker            BA         C        DD         F        GE       JPM   
date                                                                     
2018-05-02 -0.025113 -0.018046  0.032198 -0.028223  0.026602  0.004113   
2018-05-09 -0.041016  0.035478 -0.016231 -0.067737 -0.052186  0.038183   
2018-05-16  0.019178 -0.075946  0.033173 -0.018585  0.021928 -0.050763   
2018-05-23 -0.020246 -0.020514  0.009462  0.047375  0.011939  0.005813   
2018-05-30  0.024082 -0.057301  0.052816 -0.010973  0.004920 -0.058052   
...              ...       ...       ...       ...       ...       ...   
2021-03-31 -0.066483 -0.040304 -0.040181  0.033832  0.073766 -0.029731   
2021-04-07 -0.011485  0.021210 -0.028963  0.051149  0.008956  0.012997   
2021-04-14 -0.080350  0.058123 -0.014011  0.009428  0.008991  0.077935   
2021-04-21 -0.008596 -0.012307 -0.005475  0.058544  0.012861 -0.041268   
2021-04-28 -0.034981 -0.000620  0.025832 -0.004586 -0.002160  0.010118   

                                                    ...  eq_resid            \
ticker           LOW       LUV       MAR         T  ...        DD         F   
date                                                ...                       
2018-05-02  0.015891  0.028792  0.015846 -0.026628  ...       NaN       NaN   
2018-05-09  0.032949 -0.010534 -0.094166  0.000600  ... -0.005991 -0.011397   
2018-05-16  0.041921  0.002748  0.025423 -0.000250  ... -0.008350  0.016492   
2018-05-23 -0.043524  0.019622 -0.050097 -0.001256  ... -0.017731 -0.029959   
2018-05-30  0.061210  0.030632 -0.015132 -0.003654  ...  0.019622 -0.001384   
...              ...       ...       ...       ...  ...       ...       ...   
2021-03-31  0.111618  0.017395 -0.031394 -0.030551  ...  0.047641  0.037435   
2021-04-07  0.066706  0.032072 -0.012402  0.007130  ...  0.009938  0.027596   
2021-04-14 -0.023491 -0.006063 -0.033278  0.000662  ...  0.038059 -0.003320   
2021-04-21 -0.010135  0.017703 -0.008046 -0.029433  ... -0.006435  0.063250   
2021-04-28  0.057876  0.019380 -0.005763  0.008156  ... -0.005499  0.063444   

                                                                        \
ticker            GE       JPM       LOW       LUV       MAR         T   
date                                                                     
2018-05-02       NaN       NaN       NaN       NaN       NaN       NaN   
2018-05-09 -0.027041  0.022754 -0.037866  0.021486  0.023560  0.091845   
2018-05-16 -0.048740 -0.032638  0.009505  0.028936 -0.000261  0.025363   
2018-05-23 -0.031649  0.011019 -0.016197 -0.033420 -0.006868 -0.029981   
2018-05-30  0.057012  0.011225 -0.111260  0.047955  0.022784 -0.002864   
...              ...       ...       ...       ...       ...       ...   
2021-03-31  0.110701  0.017484 -0.060987  0.024703  0.031129  0.000912   
2021-04-07 -0.039561  0.024891 -0.016783 -0.008228  0.007647 -0.003580   
2021-04-14 -0.005559  0.012474  0.022144  0.017528  0.039289 -0.001811   
2021-04-21 -0.006985  0.046571 -0.020004  0.034666  0.004559  0.007111   
2021-04-28  0.040153  0.021032  0.006098  0.035002  0.039861 -0.010264   

                                
ticker           WFC       XOM  
date                            
2018-05-02       NaN       NaN  
2018-05-09  0.006137  0.032137  
2018-05-16 -0.012648 -0.018023  
2018-05-23 -0.015973 -0.025735  
2018-05-30 -0.022581  0.004106  
...              ...       ...  
2021-03-31  0.028157  0.026193  
2021-04-07  0.020143  0.074611  
2021-04-14  0.022989  0.042207  
2021-04-21 -0.025593  0.013852  
2021-04-28 -0.006496  0.050348  

[157 rows x 24 columns]

In [37]:
win_lengths = np.array([3, 6, 10, 30, 60])

In [38]:
col_list = []
for w in win_lengths:
    for s in df_resid.columns.levels[1]:
        df_pair = df_resid.loc[:, [("cds_resid", s), ("eq_resid", s)]]
        df_pair.columns = ["cds_resid", "eq_resid"]
        for win_type, df_win in {
            "exp_wm": df_pair.ewm(alpha=1 / w),
            "boxcar": df_pair.rolling(window=2 * w)
        }.items():
            df_cov = df_win.cov()
            df_mean = df_win.mean()

            s_var = df_cov["eq_resid"].xs("eq_resid", level=1)
            s_var.name = ("var_x", win_type, f"t_{w:02d}", s)

            s_cov = df_cov["eq_resid"].xs("cds_resid", level=1)
            s_cov.name = ("cov_xy", win_type, f"t_{w:02d}", s)

            s_beta_1 = s_cov / s_var
            s_beta_1.name = ("beta_1", win_type, f"t_{w:02d}", s)

            s_beta_0 = s_beta_1 * df_mean.eq_resid - df_mean.cds_resid
            s_beta_0.name = ("beta_0", win_type, f"t_{w:02d}", s)

            s_resid_sq = (s_beta_0 + df_pair.eq_resid * s_beta_1 - df_pair.cds_resid).pow(2)
            s_resid_sq.name = ("resid_sq", win_type, f"t_{w:02d}", s)

            s_error_sq = (df_pair.cds_resid.loc[~s_resid_sq.isna()] - df_pair.cds_resid.loc[~s_resid_sq.isna()].mean()).pow(2)
            s_error_sq.name = ("error_sq", win_type, f"t_{w:02d}", s)

            col_list.extend([s_resid_sq, s_error_sq, s_beta_0, s_beta_1, s_var, s_cov])

df_betas = pd.concat(col_list, axis=1)
df_betas.columns.names = ["stat", "win_type", "win_length", "ticker"]
df_betas = df_betas.stack("ticker")
df_betas = df_betas.swaplevel("stat", "win_type", axis=1)
df_betas = df_betas.sort_index(axis=1)
df_betas.tail(60)

win_type             boxcar                                                    \
stat                 beta_0                                            beta_1   
win_length             t_03      t_06      t_10      t_30      t_60      t_03   
date       ticker                                                               
2021-03-31 BA      0.054376  0.009141  0.001138  0.000035  0.006138 -0.011463   
           C       0.007960  0.000023  0.003558 -0.001399 -0.000316 -0.028721   
           DD     -0.012760 -0.005523  0.002789  0.005823  0.011121 -1.018256   
           F       0.047870  0.005800  0.008098  0.010913  0.003646  0.710110   
           GE      0.022958  0.028081  0.016034  0.007689  0.007333  0.400507   
           JPM     0.001605 -0.005888  0.004469 -0.010142 -0.006048  0.161862   
           LOW    -0.043445  0.000900  0.006154  0.006165 -0.000935 -0.930284   
           LUV    -0.001860  0.013452  0.017177  0.017138  0.009323 -0.438067   
           MAR     0.031720  0.022074  0.018311  0.000223  0.000390 -0.711282   
           T       0.012399  0.003088 -0.008481 -0.002971 -0.001902 -0.039949   
           WFC     0.004983  0.004048  0.001503  0.000854 -0.001262  0.322512   
           XOM     0.006356  0.000855  0.000218  0.007083  0.003202 -0.529701   
2021-04-07 BA      0.045833  0.006315  0.002327  0.000870  0.005934  0.025911   
           C       0.000599  0.003890  0.002560 -0.001792 -0.000574 -0.009414   
           DD     -0.013510 -0.010165 -0.003404  0.006644  0.010966 -0.975801   
           F       0.032659 -0.000239  0.003247  0.010740  0.003563  1.041016   
           GE      0.010585  0.026014  0.011109  0.007858  0.007533  0.301072   
           JPM    -0.003686 -0.000974  0.003388 -0.010879 -0.006247  0.140213   
           LOW    -0.059959 -0.012141  0.002577  0.004434 -0.001609 -0.876749   
           LUV     0.006213  0.008481  0.013285  0.017007  0.008992 -0.467141   
           MAR     0.021289  0.025177  0.012904  0.000807  0.000670 -0.692165   
           T       0.006536 -0.000060 -0.006130 -0.003124 -0.002291 -0.381030   
           WFC     0.000072  0.007643  0.000147  0.000185 -0.001687  0.256468   
           XOM     0.001890 -0.000963 -0.000637  0.007585  0.003745 -0.491687   
2021-04-14 BA      0.066624  0.020504  0.007340  0.003401  0.006249  0.243941   
           C      -0.009422 -0.003370 -0.001096 -0.002631 -0.001406  0.115077   
           DD     -0.018614 -0.013147 -0.007314  0.007378  0.009444 -0.987905   
           F       0.037688 -0.001066  0.001965  0.011326  0.003498  1.462537   
           GE      0.008332  0.023159  0.008874  0.008511  0.007600  0.317938   
           JPM    -0.021049 -0.004803 -0.002724 -0.012554 -0.006928 -0.055430   
           LOW    -0.028425 -0.013417  0.002537  0.004914 -0.000995 -1.966681   
           LUV     0.009713  0.005835  0.010148  0.016488  0.008946  0.231169   
           MAR     0.035721  0.028707  0.010189  0.001459  0.000558  0.310982   
           T      -0.000203  0.001133 -0.004912 -0.005038 -0.002273 -0.173740   
           WFC    -0.008739 -0.000215 -0.003442 -0.001464 -0.002774  0.489354   
           XOM    -0.002383 -0.003867 -0.002546  0.007533  0.003847 -0.480252   
2021-04-21 BA      0.040739  0.022281  0.011054  0.005631  0.006668 -0.329859   
           C      -0.019101 -0.002419 -0.003059 -0.002586 -0.001953 -0.351141   
           DD     -0.024291 -0.014674 -0.008399  0.005741  0.009057 -1.451913   
           F       0.028543  0.006852  0.001120  0.009712  0.002686  1.314232   
           GE      0.027050  0.022794  0.012610  0.011072  0.007017  0.486086   
           JPM    -0.061398 -0.002549 -0.006243 -0.010483 -0.007331 -1.860299   
           LOW    -0.023465 -0.009658  0.001310  0.003068 -0.000940 -1.818808   
           LUV    -0.006089 -0.003119  0.013867  0.013933  0.008869 -0.320844   
           MAR     0.048755  0.032784  0.015486 -0.000041  0.000691  0.584255   
           T       0.008588  0.

In [39]:
1 - df_betas.boxcar.resid_sq.sum() / df_betas.boxcar.error_sq.sum()

win_length
t_03   -0.316958
t_06   -0.008247
t_10    0.089397
t_30    0.079890
t_60   -0.114675
dtype: float64

In [40]:
1 - df_betas.exp_wm.resid_sq.sum() / df_betas.exp_wm.error_sq.sum()

win_length
t_03   -0.496354
t_06   -0.074866
t_10    0.033386
t_30    0.073777
t_60    0.064288
dtype: float64